In [2]:
### Tokenizer
from transformers import AutoTokenizer

In [3]:
tokenizer_eng = AutoTokenizer.from_pretrained('roberta-large')
tokenizer_eng.tokenize("this is test for [CLS]")

tokenizer_kor = AutoTokenizer.from_pretrained('klue/roberta-large')
tokenizer_kor.tokenize("this is test for [CLS]")

['this', 'is', 't', '##est', 'for', '[CLS]']

In [4]:
print(tokenizer_eng.tokenize("this is test for [CLS]"))
print(tokenizer_kor.tokenize("this is test for [CLS]"))

['this', 'Ġis', 'Ġtest', 'Ġfor', 'Ġ[', 'CL', 'S', ']']
['this', 'is', 't', '##est', 'for', '[CLS]']


In [5]:
import sys
sys.path.append('/opt/ml/level2-klue-level2-nlp-03/KSY')
from load_data import *

In [6]:
train_data_dir = "/opt/ml/dataset/train/train.csv"
# original type data
train_dataset = load_data(train_data_dir)

In [7]:
def preprocessing_ent_dataset(dataset):
  # """ 처음 불러온 csv 파일에서 sentecen에 등장하고 start_idx, end_idx 를 갖는 subject, object를 marking."""
    subject_entity = []
    object_entity = []
    subject_type = []
    object_type = []
    # subject_entity : {'word':- , 'start_idx':-, 'end_idx':-}
    for idx, (subj, obj, sents) in enumerate(zip(dataset['subject_entity'] ,dataset['object_entity'], dataset['sentence'])):
        subj, obj = eval(subj), eval(obj)
        new_subj = '[E1]' + sents[subj['start_idx'] : subj['end_idx']+1] + '[/E1]'
        new_obj = '[E2]' + sents[obj['start_idx'] : obj['end_idx']+1] + '[/E2]'
        if subj['start_idx'] >= obj['start_idx']:
            temp = sents[:obj['start_idx']] + new_obj + sents[obj['end_idx']+1 : subj['start_idx']] + new_subj + sents[subj['end_idx']+1 :]
        elif subj['start_idx'] < obj['start_idx']:
            temp = sents[:subj['start_idx']] + new_subj + sents[subj['end_idx']+1 : obj['start_idx']] + new_obj + sents[obj['end_idx']+1 :]
        
        dataset['sentence'][idx] = temp
        
        subject_entity.append(subj['word'])
        object_entity.append(obj['word'])
        
        subject_type.append(subj['type'])
        object_type.append(obj['type'])

    # print(len(dataset['id'], len(dataset['sentence']))
    out_dataset = pd.DataFrame({'id':dataset['id'], 'sentence':dataset['sentence'],
                                'subject_entity':subject_entity,'object_entity':object_entity,
                                'subject_type':subject_type,'object_type':object_type,
                                'label':dataset['label'],})
    return out_dataset

In [8]:
def en_load_data(dataset_dir):
  # """ csv 파일을 경로에 맡게 불러 옵니다. """
    pd_dataset = pd.read_csv(dataset_dir)
    dataset = preprocessing_ent_dataset(pd_dataset)
    return dataset

In [9]:
en_train_data_dir = "/opt/ml/dataset/train/train.csv"
ent_dataset = en_load_data(en_train_data_dir)

<ipython-input-7-9ebe380a9d21>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['sentence'][idx] = temp


In [10]:
from transformers import AutoTokenizer
# bert-base-uncased & bert-base-cased
sub_obj = ["[E1]", "[/E1]","[E2]", "[/E2]"]
test_tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large",
                                         add_special_tokens = True)
#dir(tokenizer)
test_tokenizer.add_special_tokens({
                            "additional_special_tokens": sub_obj
                            })

4

In [11]:
test_tokenizer.special_tokens_map

{'bos_token': '[CLS]',
 'eos_token': '[SEP]',
 'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]',
 'additional_special_tokens': ['[E1]', '[/E1]', '[E2]', '[/E2]']}

In [16]:
# tokenizer size
print(len(test_tokenizer))

32004

In [25]:
def tokenized_dataset(dataset, tokenizer):
    """ tokenizer에 따라 sentence를 tokenizing 합니다."""
    # TODO! multi sentence ; single sentence
    concat_entity = []
    for e01, e02 in zip(dataset['subject_entity'], dataset['object_entity']):
        temp = ''
        temp = e01 + '[SEP]' + e02
        concat_entity.append(temp)
    tokenized_sentences = tokenizer(
                          concat_entity,
                          list(dataset['sentence']),
                          return_tensors="pt",
                          padding=True,
                          truncation=True,
                          max_length=256,
                          add_special_tokens=True,
                          )
    return tokenized_sentences

In [12]:
out_dataset = tokenized_dataset(ent_dataset, test_tokenizer)

In [28]:
out_dataset['input_ids'][0]

tensor([    0, 29830,     2,  8373, 14113,  2234,     2,   168, 30985, 14451,
         7088,  4586,   169,   793, 32002,  8373, 14113,  2234, 32003,  1504,
         1363,  2088, 32000, 29830, 32001,   543, 14879,  2440,  6711,   170,
        21406, 26713,  2076, 25145,  5749,   171,  1421,   818,  2073,  4388,
         2062,    18,     2,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1, 

In [29]:
print(*out_dataset[1].tokens)
print(*out_dataset[1].special_tokens_mask)

[CLS] 민주 ##평 ##화 ##당 [SEP] 대안 ##신 ##당 [SEP] 호남 ##이 기반 ##인 바른 ##미 ##래 ##당 · [E2] 대안 ##신 ##당 [/E2] · [E1] 민주 ##평 ##화 ##당 [/E1] 이 우여곡절 끝 ##에 합당 ##해 민생 ##당 ( 가칭 ) 으로 재 ##탄 ##생 ##한다 . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [39]:
out_list = out_dataset[1].tokens
print(out_list)
for idx, c in enumerate(out_list):
    if c =='[PAD]':
        print(idx)
        break

['[CLS]', '민주', '##평', '##화', '##당', '[SEP]', '대안', '##신', '##당', '[SEP]', '호남', '##이', '기반', '##인', '바른', '##미', '##래', '##당', '·', '[E2]', '대안', '##신', '##당', '[/E2]', '·', '[E1]', '민주', '##평', '##화', '##당', '[/E1]', '이', '우여곡절', '끝', '##에', '합당', '##해', '민생', '##당', '(', '가칭', ')', '으로', '재', '##탄', '##생', '##한다', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '

In [43]:
d_data = out_dataset[1].attention_mask
for idx, d in enumerate(d_data):
    if d ==0:
        print(idx)
        break

49


In [14]:
out_dataset['token_type_ids'][0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])

In [16]:
#bert-base-uncased
test_tokenizer2 = AutoTokenizer.from_pretrained("klue/bert-base",
                                         add_special_tokens = True)

In [17]:
out_dataset = tokenized_dataset(ent_dataset, test_tokenizer2)

In [18]:
out_dataset['token_type_ids'][0]

tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])